In [1]:
import logging
import os
import pdb
import warnings
from pathlib import Path

import metpy.constants
import numpy as np
import pandas as pd
import seaborn as sns
import xarray
from matplotlib.colors import ListedColormap
from metpy.units import units
from sklearn.neighbors import BallTree
from tqdm import tqdm

from hwt2024 import Model, firstRun, helicityThresholds, windThresholds

sns.set_theme()
logging.basicConfig(level=logging.WARNING, format="%(asctime)s %(message)s", force=True)
tmpdir = Path(os.getenv("TMPDIR"))

In [6]:
def assigncoords(ds: xarray.Dataset):
    """
    Assign member number, forecast hour,
    initialiation time, and valid time to xarray Dataset.
    - Read member number from component of file path
    - Get forecast hour from global attribute (.attrs["forecastHour"])
    - init time comes from global attribute
    - valid time derived from init time and forecast hour.
    """

    # get member number from original name of file, which is
    # held in Dataset.encoding["source"].
    filename = Path(ds.encoding["source"])
    # grab part that starts with "mem"
    mem = [p for p in filename.parts if p.startswith("mem")]
    # strip off the "mem_" part and keep the reset
    mem = mem[0].lstrip("mem_")
    # convert to integer
    mem = int(mem)

    # read forecastHour from global attribute
    forecastHour = ds.attrs["forecastHour"]
    forecastHour = float(forecastHour)

    # assign dayForecast (after forecastHour is defined)
    # add 11 instead of 12 because we want forecastHours 13-36 to map to day 1
    # because uh max covers the previous hour. so does 10-m wind max
    # forecastHours 37-60 map to day 2
    dayForecast = int((forecastHour + 11) / 24)

    # read initializationTime from global attribute
    initializationTime = ds.attrs["initializationTime"]
    initializationTime = pd.to_datetime(initializationTime, format="%Y%m%d%H")

    #  valid_time = initializationTime + forecastHour
    valid_time = initializationTime + pd.to_timedelta(forecastHour, unit="hour")

    # I brought these assignments down here together to see if it is faster.
    # assign to coordinate
    ds = ds.assign_coords(mem=mem, dayForecast=dayForecast)
    # thought assign_coords would add dim to data_vars but it didn't
    ds = ds.expand_dims(dim=["mem", "dayForecast"])
    # no square brackets around initializationTime so it doesn't become a coordinate.
    # ds = ds.assign(initializationTime=initializationTime)
    ds = ds.assign(
        forecastHour=forecastHour,
        initializationTime=initializationTime,
        valid_time=[valid_time],
    )
    return ds

In [7]:
group = "wind"
thresholds = helicityThresholds if group == "uh" else windThresholds
genericnames = ["updraft max", "10m speed max"]
if group == "uh":
    genericnames = ["0-1km UH", "0-3km UH", "2-5km UH"]
fv3 = Model(
    "fv3",
    nmem=10,
    v=["MXUPHL0_1km_max", "MXUPHL0_3km_max", "MXUPHL2_5km_max"] if group == "uh" else ["MAXUVV_max", "MAXWIND10m"],
    lead_time_days=8,
)
mpas = Model(
    "mpas",
    nmem=5,
    v=["updraft_helicity_max01", "updraft_helicity_max03", "updraft_helicity_max"] if group == "uh" else ["w_velocity_max", "wind_speed_10m_max"],
    lead_time_days=5,
)
models = [fv3, mpas]
# assert models all have same number of variables
nvars = len(genericnames)
assert all(
    [len(model.v) == nvars for model in models]
), "model objects have different number of variables to analyze"
forecast_hours = range(13,37)


In [5]:
example = "/glade/campaign/mmm/parc/schwartz/HWT2024/mpas/2024052100/post/mem_1/interp_mpas_3km_2024052100_mem1_f018.nc"
ds = xarray.open_dataset(example).squeeze()  # squeeze 1-element time dimension
latitudes = ds.latitude
longitudes = ds.longitude

In [16]:
client.cluster.close()
client.shutdown()
client.close()

2024-07-11 18:21:50,639 Closing Nanny at 'tcp://127.0.0.1:41827'. Reason: nanny-close
2024-07-11 18:21:50,640 Nanny asking worker to close. Reason: nanny-close
2024-07-11 18:21:50,641 Closing Nanny at 'tcp://127.0.0.1:46257'. Reason: nanny-close
2024-07-11 18:21:50,642 Nanny asking worker to close. Reason: nanny-close
2024-07-11 18:21:50,642 Closing Nanny at 'tcp://127.0.0.1:34337'. Reason: nanny-close
2024-07-11 18:21:50,643 Nanny asking worker to close. Reason: nanny-close
2024-07-11 18:21:50,643 Closing Nanny at 'tcp://127.0.0.1:43249'. Reason: nanny-close
2024-07-11 18:21:50,644 Nanny asking worker to close. Reason: nanny-close
2024-07-11 18:21:50,644 Closing Nanny at 'tcp://127.0.0.1:39345'. Reason: nanny-close
2024-07-11 18:21:50,645 Nanny asking worker to close. Reason: nanny-close
2024-07-11 18:21:50,645 Closing Nanny at 'tcp://127.0.0.1:41917'. Reason: nanny-close
2024-07-11 18:21:50,646 Nanny asking worker to close. Reason: nanny-close
2024-07-11 18:21:50,647 Closing Nanny at

In [17]:
# worked well on casper batch with 32 CPUs (36 possible but took long time in queue)
# and 30G memory
from dask.distributed import Client

client = Client(n_workers=70)
client

2024-07-11 18:22:09,122 State start
2024-07-11 18:22:09,588   Scheduler at:     tcp://127.0.0.1:38415
2024-07-11 18:22:09,589   dashboard at:  https://jupyterhub.hpc.ucar.edu/stable/user/ahijevyc/proxy/8787/status
2024-07-11 18:22:09,589 Registering Worker plugin shuffle
2024-07-11 18:22:10,165         Start Nanny at: 'tcp://127.0.0.1:35505'
2024-07-11 18:22:10,168         Start Nanny at: 'tcp://127.0.0.1:46129'
2024-07-11 18:22:10,171         Start Nanny at: 'tcp://127.0.0.1:36299'
2024-07-11 18:22:10,174         Start Nanny at: 'tcp://127.0.0.1:42761'
2024-07-11 18:22:10,178         Start Nanny at: 'tcp://127.0.0.1:39203'
2024-07-11 18:22:10,181         Start Nanny at: 'tcp://127.0.0.1:46761'
2024-07-11 18:22:10,185         Start Nanny at: 'tcp://127.0.0.1:36195'
2024-07-11 18:22:10,189         Start Nanny at: 'tcp://127.0.0.1:42737'
2024-07-11 18:22:10,192         Start Nanny at: 'tcp://127.0.0.1:38043'
2024-07-11 18:22:10,196         Start Nanny at: 'tcp://127.0.0.1:43103'
2024-07-

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/ahijevyc/proxy/8787/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/ahijevyc/proxy/8787/status,Workers: 70
Total threads: 280,Total memory: 235.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:38415,Workers: 70
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/ahijevyc/proxy/8787/status,Total threads: 280
Started: Just now,Total memory: 235.00 GiB
Comm: tcp://127.0.0.1:41963,Total threads: 4
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/ahijevyc/proxy/37715/status,Memory: 3.36 GiB
Nanny: tcp://127.0.0.1:35505,


In [18]:
d2023 = pd.date_range(start=firstRun(2023), end="20230531", freq="1D")
d2024 = pd.date_range(start=firstRun(2024), end="20240531", freq="1D")
for valid_date in d2023.union(d2024):
    rptfile = tmpdir / f"near_rpt.{valid_date}.nc"
    if os.path.exists(rptfile):
        logging.warning(f"open existing {rptfile}")
        near_rpts = xarray.open_dataarray(rptfile)
    else:
        logging.warning(f"create new {rptfile}")
        # Load severe weather reports
        near_rpts = []
        rpt_types = ["torn", "wind", "hail"]
        for rpt_type in rpt_types:
            rpt_dist_thresh_miles = 25 * units.miles
            reports = pd.read_csv(
                f"https://www.spc.noaa.gov/climo/reports/{valid_date.strftime('%y%m%d')}_rpts_{rpt_type}.csv"
            )
            print(f"read {len(reports)} {rpt_type} reports {valid_date}")

            logging.info("Create DataArray like latitudes with all elements set to False")
            near_rpt = xarray.full_like(latitudes, False, dtype=bool)
            near_rpt.name = rpt_type
            near_rpt.attrs["range"] = str(rpt_dist_thresh_miles)

            # If at least one report, put True in neighboring values
            if not reports.empty:
                # Prepare wind report coordinates for spatial indexing
                rpt_coords = reports[["Lat", "Lon"]]

                # Prepare model grid coordinates for spatial indexing
                # .ravel reads 2-d array as if it were a 1-d array.
                uh_coords = np.c_[latitudes.values.ravel(), longitudes.values.ravel()]

                # BallTree (with metric = "haversine") assumes spherical coordinates
                uh_tree = BallTree(np.deg2rad(uh_coords), metric="haversine")

                Re = metpy.constants.earth_avg_radius
                r = rpt_dist_thresh_miles.to("km") / Re
                r = r.to_base_units()
                # Find all uh points whose distance is at most threshold_distance from wind report
                results = uh_tree.query_radius(np.deg2rad(rpt_coords), r=r)
                for result in results:
                    near_rpt.values.put(result, True)

            near_rpts.append(near_rpt)
        # tried concat with dim argument but it didn't preserve coord labels
        near_rpts = xarray.merge(near_rpts).to_dataarray(dim="rpt_type", name="near_rpts")
        near_rpts.to_netcdf(rptfile)
    for model in models:
        idir = Path(f"/glade/campaign/mmm/parc/schwartz/HWT{valid_date.strftime('%Y')}/{model}")
        ncfile = tmpdir / f"forecast_yes.{model}.{thresholds.attrs['short_name']}.{valid_date.strftime('%Y%m%d')}.nc"
        if False and os.path.exists(ncfile) and os.stat(ncfile).st_size > 50000:
            logging.warning(f"skip existing {ncfile}")
            continue
        else:
            logging.warning(f"create new {ncfile}")
            # Load updraft helicity data from netCDF
            fmt = "%Y%m%d%H"
            oneday = pd.to_timedelta(1, unit="day")

            # Create list of input files
            # This is a nested list comprehension, looping through
            # forecast_hours (iterable of forecast hours)
            #    lead_time_days (iterable of lead times in days)
            #        members (1 through model.nmem)
            ifiles = [
                idir
                / (valid_date - lead_time_day * oneday).strftime(fmt)
                / "post"
                / f"mem_{mem}"
                / f"interp_{model}_3km_{(valid_date-lead_time_day*oneday).strftime(fmt)}_mem{mem}_f{fhr+lead_time_day*24:03d}.nc"
                for mem in range(1, model.nmem + 1)
                for lead_time_day in range(model.lead_time_days)
                for fhr in forecast_hours
            ]
            ifiles = [f for f in ifiles if os.path.exists(f)]
            print(f"open {len(ifiles)} files")
            print(ifiles[0:8])
            assert len(ifiles) % 24 == 0, "# ifiles should be multiple of 24"
            ds0 = xarray.open_mfdataset(
                ifiles,
                preprocess=assigncoords,
                drop_variables=["total_precip_hrly"],
                combine_attrs="drop",
                compat="override",
                coords="minimal",
                parallel=True,
                decode_cf=False,
                decode_coords=False,
                engine="h5netcdf",  # helped with dask and HDF/lock errors
            )

            # remove time dimension
            ds0 = ds0.squeeze(dim="time", drop=True)

            # Extract updraft helicity values.
            # Reduce all valid_times
            # Reduce all members to ensemble maximum.
            logging.warning(model.v)

            v = ds0[model.v].max(dim=["mem", "valid_time"]).to_dataarray()  # merge variables into "variable" dimension
            # multiple thresholds
            fy = xarray.concat(
                [v >= t for t in thresholds],
                dim="thresh",
            ).assign_coords(thresh=thresholds)
            fy.name = "ensmax"

            v = ds0[model.v].max(dim="valid_time").to_dataarray()  # merge variables into "variable" dimension
            # multiple thresholds
            nfy = xarray.concat(
                [(v >= t).sum(dim="mem") for t in thresholds],
                dim="thresh",
            ).assign_coords(thresh=thresholds)
            nfy.name = "enssum"

            ds = xarray.merge([fy, nfy]) # don't lose units and shortname attributes of thresholds, combine_attrs="drop")
            ds.attrs["model"] = str(model)
            ds.attrs["nmem"] = model.nmem
            ds.to_netcdf(ncfile)
            logging.warning(f"saved {ncfile}")

2024-07-11 18:22:17,671 open existing /glade/derecho/scratch/ahijevyc/tmp/near_rpt.2023-04-24 00:00:00.nc
2024-07-11 18:22:17,680 create new /glade/derecho/scratch/ahijevyc/tmp/forecast_yes.fv3.wind.20230424.nc


open 240 files
[PosixPath('/glade/campaign/mmm/parc/schwartz/HWT2023/fv3/2023042400/post/mem_1/interp_fv3_3km_2023042400_mem1_f013.nc'), PosixPath('/glade/campaign/mmm/parc/schwartz/HWT2023/fv3/2023042400/post/mem_1/interp_fv3_3km_2023042400_mem1_f014.nc'), PosixPath('/glade/campaign/mmm/parc/schwartz/HWT2023/fv3/2023042400/post/mem_1/interp_fv3_3km_2023042400_mem1_f015.nc'), PosixPath('/glade/campaign/mmm/parc/schwartz/HWT2023/fv3/2023042400/post/mem_1/interp_fv3_3km_2023042400_mem1_f016.nc'), PosixPath('/glade/campaign/mmm/parc/schwartz/HWT2023/fv3/2023042400/post/mem_1/interp_fv3_3km_2023042400_mem1_f017.nc'), PosixPath('/glade/campaign/mmm/parc/schwartz/HWT2023/fv3/2023042400/post/mem_1/interp_fv3_3km_2023042400_mem1_f018.nc'), PosixPath('/glade/campaign/mmm/parc/schwartz/HWT2023/fv3/2023042400/post/mem_1/interp_fv3_3km_2023042400_mem1_f019.nc'), PosixPath('/glade/campaign/mmm/parc/schwartz/HWT2023/fv3/2023042400/post/mem_1/interp_fv3_3km_2023042400_mem1_f020.nc')]


2024-07-11 18:23:04,268 Event loop was unresponsive in Nanny for 3.51s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
2024-07-11 18:23:04,325 Event loop was unresponsive in Nanny for 3.57s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
2024-07-11 18:23:04,392 Event loop was unresponsive in Nanny for 3.64s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
2024-07-11 18:23:04,418 Event loop was unresponsive in Nanny for 3.66s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
2024-07-11 18:23:04,419 Event loop was unresponsive in Nanny for 3.66s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause t

open 120 files
[PosixPath('/glade/campaign/mmm/parc/schwartz/HWT2023/mpas/2023042400/post/mem_1/interp_mpas_3km_2023042400_mem1_f013.nc'), PosixPath('/glade/campaign/mmm/parc/schwartz/HWT2023/mpas/2023042400/post/mem_1/interp_mpas_3km_2023042400_mem1_f014.nc'), PosixPath('/glade/campaign/mmm/parc/schwartz/HWT2023/mpas/2023042400/post/mem_1/interp_mpas_3km_2023042400_mem1_f015.nc'), PosixPath('/glade/campaign/mmm/parc/schwartz/HWT2023/mpas/2023042400/post/mem_1/interp_mpas_3km_2023042400_mem1_f016.nc'), PosixPath('/glade/campaign/mmm/parc/schwartz/HWT2023/mpas/2023042400/post/mem_1/interp_mpas_3km_2023042400_mem1_f017.nc'), PosixPath('/glade/campaign/mmm/parc/schwartz/HWT2023/mpas/2023042400/post/mem_1/interp_mpas_3km_2023042400_mem1_f018.nc'), PosixPath('/glade/campaign/mmm/parc/schwartz/HWT2023/mpas/2023042400/post/mem_1/interp_mpas_3km_2023042400_mem1_f019.nc'), PosixPath('/glade/campaign/mmm/parc/schwartz/HWT2023/mpas/2023042400/post/mem_1/interp_mpas_3km_2023042400_mem1_f020.nc')]


2024-07-11 18:23:32,937 ['w_velocity_max', 'wind_speed_10m_max']
2024-07-11 18:23:32,942 Event loop was unresponsive in Nanny for 3.15s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
2024-07-11 18:23:32,943 Event loop was unresponsive in Nanny for 3.15s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
2024-07-11 18:23:32,943 Event loop was unresponsive in Nanny for 3.15s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
2024-07-11 18:23:32,944 Event loop was unresponsive in Nanny for 3.15s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
2024-07-11 18:23:32,944 Event loop was unresponsive in Nanny for 3.15s.  This is often caused by long-running GIL-h

open 480 files
[PosixPath('/glade/campaign/mmm/parc/schwartz/HWT2023/fv3/2023042500/post/mem_1/interp_fv3_3km_2023042500_mem1_f013.nc'), PosixPath('/glade/campaign/mmm/parc/schwartz/HWT2023/fv3/2023042500/post/mem_1/interp_fv3_3km_2023042500_mem1_f014.nc'), PosixPath('/glade/campaign/mmm/parc/schwartz/HWT2023/fv3/2023042500/post/mem_1/interp_fv3_3km_2023042500_mem1_f015.nc'), PosixPath('/glade/campaign/mmm/parc/schwartz/HWT2023/fv3/2023042500/post/mem_1/interp_fv3_3km_2023042500_mem1_f016.nc'), PosixPath('/glade/campaign/mmm/parc/schwartz/HWT2023/fv3/2023042500/post/mem_1/interp_fv3_3km_2023042500_mem1_f017.nc'), PosixPath('/glade/campaign/mmm/parc/schwartz/HWT2023/fv3/2023042500/post/mem_1/interp_fv3_3km_2023042500_mem1_f018.nc'), PosixPath('/glade/campaign/mmm/parc/schwartz/HWT2023/fv3/2023042500/post/mem_1/interp_fv3_3km_2023042500_mem1_f019.nc'), PosixPath('/glade/campaign/mmm/parc/schwartz/HWT2023/fv3/2023042500/post/mem_1/interp_fv3_3km_2023042500_mem1_f020.nc')]


2024-07-11 18:24:27,979 Event loop was unresponsive in Nanny for 3.02s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
2024-07-11 18:24:29,079 Event loop was unresponsive in Nanny for 4.12s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
2024-07-11 18:24:31,569 Event loop was unresponsive in Nanny for 6.61s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
2024-07-11 18:24:31,570 Event loop was unresponsive in Nanny for 6.61s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
2024-07-11 18:24:31,571 Event loop was unresponsive in Nanny for 6.61s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause t

ValueError: cannot find dimension variable in this group or parent groups